In [11]:
import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import lil_matrix
from multiprocessing import Pool, cpu_count
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss,
)

In [12]:
cn_path = "/home/snu/Downloads/Histogram_CN_FA_20bin_updated.npy"
mci_path = "/home/snu/Downloads/Histogram_MCI_FA_20bin_updated.npy"
k_neighbors = 20
random_state = 42

In [13]:
def cluster_accuracy(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_sum_assignment(w.max() - w)
    ind = np.array(ind).T
    return sum([w[i, j] for i, j in ind]) / y_pred.size

In [14]:
def hungarian_map(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_sum_assignment(w.max() - w)
    ind = np.array(ind).T
    new_pred = np.zeros_like(y_pred, dtype=np.int64)
    for i, j in ind:
        new_pred[y_pred == i] = j
    return new_pred

In [15]:
def compute_similarity_for_points(points, data, neighbors, max_dis):
    n = data.shape[0]
    local_similarity_matrix = lil_matrix((n, n), dtype=np.float32)
    neighbor_sets = {i: set(neighbors[i]) for i in points}
    for i in points:
        i_neighbors = neighbor_sets[i]
        point_i = data[i]
        for j in neighbors[i]:
            if j in neighbor_sets:
                j_neighbors = neighbor_sets[j]
            else:
                j_neighbors = set(neighbors[j])
            shared_neighbors = i_neighbors & j_neighbors
            if shared_neighbors:
                shared_idx = list(shared_neighbors)
                shared_points = data[shared_idx]
                point_j = data[j]
                d_i = np.linalg.norm(shared_points - point_i[np.newaxis, :], axis=1) / (max_dis + 1e-12)
                d_j = np.linalg.norm(shared_points - point_j[np.newaxis, :], axis=1) / (max_dis + 1e-12)
                d = 0.5 * (d_i + d_j)
                similarity = np.sum(np.exp(-d * d))
                if similarity > 0:
                    local_similarity_matrix[i, j] = similarity
    return local_similarity_matrix

In [16]:
def compute_similarity(data, k):
    n = data.shape[0]
    nn_model = NearestNeighbors(n_neighbors=k, algorithm='auto')
    nn_model.fit(data)
    distances, neighbors = nn_model.kneighbors(data)
    max_dis = np.max(distances) if distances.size else 1.0
    num_processes = max(1, cpu_count() - 1)
    points_split = np.array_split(range(n), num_processes)
    args = [(points, data, neighbors, max_dis) for points in points_split]
    with Pool(processes=num_processes) as pool:
        results = pool.starmap(compute_similarity_for_points, args)
    similarity_matrix = results[0]
    for mat in results[1:]:
        similarity_matrix = similarity_matrix + mat
    similarity_matrix = similarity_matrix.maximum(similarity_matrix.transpose())
    if similarity_matrix.data.size > 0:
        similarity_matrix.data = similarity_matrix.data / similarity_matrix.max()
    similarity_matrix.setdiag(1.0 + 1e-15)
    return similarity_matrix.tocsr()

In [17]:
def compute_threshold_matrix(data, k):
    n = data.shape[0]
    similarity_matrix = compute_similarity(data, k)
    density = np.zeros(n, dtype=np.float32)
    top_k_indices = []
    for i in range(n):
        start, end = similarity_matrix.indptr[i], similarity_matrix.indptr[i+1]
        row_data = similarity_matrix.data[start:end]
        row_idx = similarity_matrix.indices[start:end]
        if row_data.size == 0:
            top_k_indices.append(np.array([], dtype=int))
            continue
        order = np.argsort(-row_data)
        sorted_idx = row_idx[order]
        sorted_data = row_data[order]
        k_eff = min(k, sorted_data.size)
        density[i] = np.sum(sorted_data[:k_eff])
        top_k_indices.append(sorted_idx)
    if density.max() > 0:
        density = density / density.max()
    nearest_neighbor_ranks = np.full(n, -1, dtype=int)
    for i in range(n):
        cur = density[i]
        for rank, nb in enumerate(top_k_indices[i]):
            if density[nb] > cur:
                nearest_neighbor_ranks[i] = rank
                break
    leader_points = np.full(n, -1, dtype=int)
    degree = density.copy()
    max_rank = int(nearest_neighbor_ranks.max()) if nearest_neighbor_ranks.max() >= 0 else 1
    sorted_by_density_indices = np.argsort(density)
    for i in sorted_by_density_indices:
        if nearest_neighbor_ranks[i] != -1:
            neighbor_idx = top_k_indices[i][nearest_neighbor_ranks[i]]
            contribution = degree[i] * np.exp(- (float(nearest_neighbor_ranks[i]) / float(max_rank))**2)
            degree[neighbor_idx] += contribution
    for i in range(n):
        if nearest_neighbor_ranks[i] != -1:
            neighbor_idx = top_k_indices[i][nearest_neighbor_ranks[i]]
            if degree[i] < degree[neighbor_idx]:
                leader_points[i] = neighbor_idx
    core_points = np.where(leader_points == -1)[0]
    core_idx_mapping = np.full(n, -1, dtype=int)
    core_idx_mapping[core_points] = np.arange(core_points.shape[0], dtype=int)
    visited = np.zeros(n, dtype=bool)
    for i in range(n):
        if visited[i]:
            continue
        if leader_points[i] == -1:
            leader_points[i] = i
            visited[i] = True
            continue
        cur = i
        stack = []
        while leader_points[cur] != -1 and leader_points[cur] != cur:
            stack.append(cur)
            visited[cur] = True
            cur = leader_points[cur]
        if leader_points[cur] == -1:
            leader_points[cur] = cur
        visited[cur] = True
        core = cur
        while stack:
            node = stack.pop()
            leader_points[node] = core
    S_coo = similarity_matrix.tocoo()
    rows, cols, vals = S_coo.row, S_coo.col, S_coo.data
    mask = rows < cols
    rows, cols, vals = rows[mask], cols[mask], vals[mask]
    weights = vals * density[rows] * density[cols]
    core_i = leader_points[rows]
    core_j = leader_points[cols]
    inter_mask = core_i != core_j
    core_i = core_i[inter_mask]
    core_j = core_j[inter_mask]
    weights = weights[inter_mask]
    core_i_mapped = core_idx_mapping[core_i]
    core_j_mapped = core_idx_mapping[core_j]
    valid_mask = (core_i_mapped >= 0) & (core_j_mapped >= 0)
    core_i_mapped = core_i_mapped[valid_mask].astype(int)
    core_j_mapped = core_j_mapped[valid_mask].astype(int)
    weights = weights[valid_mask]
    edges = list(zip(weights.tolist(), core_i_mapped.tolist(), core_j_mapped.tolist()))
    edges.sort(reverse=True, key=lambda x: x[0])
    m = core_points.shape[0]
    if m == 0:
        return np.zeros((0, 0), dtype=np.float32), leader_points, core_idx_mapping
    threshold_matrix = np.zeros((m, m), dtype=np.float32)
    core_labels = np.arange(m, dtype=int)
    for sim, i, j in edges:
        if core_labels[i] != core_labels[j]:
            label_i = core_labels[i]
            label_j = core_labels[j]
            comp_i = (core_labels == label_i)
            comp_j = (core_labels == label_j)
            threshold_matrix[np.ix_(comp_i, comp_j)] = sim
            core_labels[comp_i] = label_j
    threshold_matrix = np.maximum(threshold_matrix, threshold_matrix.T)
    np.fill_diagonal(threshold_matrix, 1.0 + 1e-15)
    return threshold_matrix, leader_points, core_idx_mapping

In [18]:
def tango(data, cluster_num, k, run_seed=None):
    threshold_matrix, leader_points, core_idx_mapping = compute_threshold_matrix(data, k)
    rng = run_seed if run_seed is not None else None
    if threshold_matrix.size == 0 or threshold_matrix.shape[0] < cluster_num:
        S_full = compute_similarity(data, k).toarray()
        clustering = SpectralClustering(
            n_clusters=cluster_num,
            affinity='precomputed',
            assign_labels='kmeans',
            random_state=run_seed
        )
        labels_full = clustering.fit_predict(S_full)
        return labels_full
    clustering = SpectralClustering(
        n_clusters=cluster_num,
        affinity='precomputed',
        assign_labels='kmeans',
        random_state=run_seed
    )
    core_labels = clustering.fit_predict(threshold_matrix)
    labels_full = core_labels[core_idx_mapping[leader_points]]
    return labels_full

In [20]:
def main():
    CN = np.load(cn_path, allow_pickle=True)
    MCI = np.load(mci_path, allow_pickle=True)
    X_cn = np.array(list(CN))
    X_mci = np.array(list(MCI))
    X = np.vstack([X_cn, X_mci]).astype(np.float32)
    y = np.hstack([np.zeros(X_cn.shape[0], dtype=np.int64), np.ones(X_mci.shape[0], dtype=np.int64)])
    np.random.seed(random_state)
    perm = np.random.permutation(X.shape[0])
    X = X[perm]
    y = y[perm]
    y_pred = tango(X, cluster_num=len(np.unique(y)), k=k_neighbors)
    y_pred_aligned = hungarian_map(y, y_pred)
    n_classes = len(np.unique(y))
    y_proba = np.zeros((len(y), n_classes), dtype=float)
    for idx, lab in enumerate(y_pred_aligned):
        if 0 <= lab < n_classes:
            y_proba[idx, lab] = 1.0
        else:
            y_proba[idx, :] = 1.0 / n_classes
    acc = accuracy_score(y, y_pred_aligned)
    if n_classes == 2:
        acc_inv = accuracy_score(y, 1 - y_pred_aligned)
        if acc_inv > acc:
            acc = acc_inv
            y_pred_aligned = 1 - y_pred_aligned
    prec = precision_score(y, y_pred_aligned, zero_division=0)
    rec = recall_score(y, y_pred_aligned, zero_division=0)
    f1 = f1_score(y, y_pred_aligned, zero_division=0)
    ll = log_loss(y, y_proba)
    print("==== TANGO results on your features ====")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1: {f1:.4f}")
    print(f"LogLoss: {ll:.6f}")

if __name__ == "__main__":
    main()

==== TANGO results on your features ====
Accuracy: 0.6167
Precision: 0.5956
Recall: 0.9701
F1: 0.7380
LogLoss: 13.816734


In [22]:
if __name__ == "__main__":
    import torch, random

    num_runs = 10
    all_results = {"accuracy": [], "precision": [], "recall": [], "f1": [], "log_loss": []}

    for run in range(num_runs):
        print(f"\n--- Run {run+1}/{num_runs} ---")
        torch.manual_seed(run)
        np.random.seed(run)
        random.seed(run)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(run)

        CN = np.load(cn_path, allow_pickle=True)
        MCI = np.load(mci_path, allow_pickle=True)
        X_cn = np.array(list(CN))
        X_mci = np.array(list(MCI))
        X = np.vstack([X_cn, X_mci]).astype(np.float32)
        y = np.hstack([np.zeros(X_cn.shape[0], dtype=np.int64), np.ones(X_mci.shape[0], dtype=np.int64)])
        np.random.seed(random_state)
        perm = np.random.permutation(X.shape[0])
        X = X[perm]
        y = y[perm]
        y_pred = tango(X, cluster_num=len(np.unique(y)), k=k_neighbors)
        y_pred_aligned = hungarian_map(y, y_pred)
        n_classes = len(np.unique(y))
        y_proba = np.zeros((len(y), n_classes), dtype=float)
        for idx, lab in enumerate(y_pred_aligned):
            if 0 <= lab < n_classes:
                y_proba[idx, lab] = 1.0
            else:
                y_proba[idx, :] = 1.0 / n_classes
        acc = accuracy_score(y, y_pred_aligned)
        if n_classes == 2:
            acc_inv = accuracy_score(y, 1 - y_pred_aligned)
            if acc_inv > acc:
                acc = acc_inv
                y_pred_aligned = 1 - y_pred_aligned
        prec = precision_score(y, y_pred_aligned, zero_division=0)
        rec = recall_score(y, y_pred_aligned, zero_division=0)
        f1 = f1_score(y, y_pred_aligned, zero_division=0)
        ll = log_loss(y, y_proba)

        print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}, LogLoss: {ll:.6f}")

        all_results["accuracy"].append(acc)
        all_results["precision"].append(prec)
        all_results["recall"].append(rec)
        all_results["f1"].append(f1)
        all_results["log_loss"].append(ll)

    print("\n================ FINAL SUMMARY ================\n")
    print(f"{'Metric':>12} | {'Mean':>10} ± {'Std':<10}")
    print("-" * 40)
    for metric, values in all_results.items():
        print(f"{metric:>12} | {np.mean(values):10.4f} ± {np.std(values):<10.4f}")


--- Run 1/10 ---
Accuracy: 0.6167, Precision: 0.5956, Recall: 0.9701, F1: 0.7380, LogLoss: 13.816734

--- Run 2/10 ---
Accuracy: 0.6167, Precision: 0.5956, Recall: 0.9701, F1: 0.7380, LogLoss: 13.816734

--- Run 3/10 ---
Accuracy: 0.6167, Precision: 0.5956, Recall: 0.9701, F1: 0.7380, LogLoss: 13.816734

--- Run 4/10 ---
Accuracy: 0.6167, Precision: 0.5956, Recall: 0.9701, F1: 0.7380, LogLoss: 13.816734

--- Run 5/10 ---
Accuracy: 0.6167, Precision: 0.5956, Recall: 0.9701, F1: 0.7380, LogLoss: 13.816734

--- Run 6/10 ---
Accuracy: 0.6167, Precision: 0.5956, Recall: 0.9701, F1: 0.7380, LogLoss: 13.816734

--- Run 7/10 ---
Accuracy: 0.6167, Precision: 0.5956, Recall: 0.9701, F1: 0.7380, LogLoss: 13.816734

--- Run 8/10 ---
Accuracy: 0.6167, Precision: 0.5956, Recall: 0.9701, F1: 0.7380, LogLoss: 13.816734

--- Run 9/10 ---
Accuracy: 0.6167, Precision: 0.5956, Recall: 0.9701, F1: 0.7380, LogLoss: 13.816734

--- Run 10/10 ---
Accuracy: 0.6167, Precision: 0.5956, Recall: 0.9701, F1: 0.7380